In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import math
import numpy
import os
import re
from datetime import datetime
import math

Debemos primero crear un diccionario que apunte a los links o rutas que se emplean para subir la informacion

In [ ]:
folder_base = r"C:\Users\dorito\OneDrive - Coocentral\Reportes Qfield\2. ADRIAN OSPINA RIOS\2908"
folder_individuales = os.path.join(folder_base, "shp_individuales")
folder_agrupados = os.path.join(folder_base, "shp_agrupados")
url_base = "https://192.168.179.3/"
capas_gdf_dict = {
    "Finca.shp": f"{url_base}api/v1/poligonos_fincas/",
    "Lote.shp": f"{url_base}api/v1/poligonos_lotes/",
    "Construcciones.shp": f"{url_base}api/v1/poligonos_infraestructura/",
    "Conservacion.shp": f"{url_base}api/v1/poligonos_conservacion/"
}
capas_gdf_dict_agrupados = {
    "Fincas.shp": f"{url_base}api/v1/poligonos_fincas/",
    "Lotes_Mancipe.shp": f"{url_base}api/v1/poligonos_lotes/",
    "Lotes_Otros.shp": f"{url_base}api/v1/poligonos_lotes/",
}



In [ ]:
login = {
    "username":"Dorito",
    "password":"Portador123"
}
# Crear una sesión para mantener la cookie
session = requests.Session()
response = session.post(f"{url_base}api/user/login/", login, verify=False)

In [ ]:
files_individuales = [(archivo, os.path.join(folder_individuales, archivo))for archivo in os.listdir(folder_individuales) if archivo.endswith(".shp")]
files_agrupados = [(archivo, os.path.join(folder_agrupados, archivo))for archivo in os.listdir(folder_agrupados) if archivo.endswith(".shp")]

In [ ]:
def limpiar_nan(obj):
    if isinstance(obj, dict):
        return {k: limpiar_nan(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [limpiar_nan(v) for v in obj]
    elif isinstance(obj, float) and (math.isnan(obj) or math.isinf(obj)):
        return None
    else:
        return obj
    

def reportar_lotes(row):
    if pd.notna(row["doc_aso"]):
        documento = int(row["doc_aso"])
    elif pd.notna(row["doc_prod"]):
        documento = int(row["doc_prod"])
    else:
        documento = None
    response = {
        "documento_productor":documento,
        "poligono":str(row["geometry"]),
        "mongo_atribute":{
            "numero_documento": str(documento),
            "nombre_productor": str(row['nom_prod']),
            "fecha_visita": pd.to_datetime(row['fecha_vis']).strftime("%Y-%m-%d") if pd.notnull(row['fecha_vis']) else None,
            "area": row['area'],
            "observaciones": "",
            "descripcion": "",
            "numero_lote": row['num_Lote'],
            "descripcion_lote": "",
            "variedad": row['variedad'],
            "distancia_surcos": row['dist_surco'],
            "distancia_plantas": row['dist_plant'],
            "densidad": row['densidad'],
            "numero_plantas": row['num_planta'],
            "gramos_plantas": row['gr_plant'],
            "kg_produccion": row['kg_produc'],
            "fecha_actividad": pd.to_datetime(row['fecha_acti']).strftime("%Y-%m-%d") if pd.notnull(row['fecha_acti']) else None,
            "produccion": row['produccion'],
            "estado_cultivo":row['estado_cul'],
            "subtipo_operacion": row['subtipo_op']
        }
    }
    response = limpiar_nan(response)
    return response

def reportar_fincas(row):
    if bool(row['Asociado']):
        if not pd.isna(row["doc_prod"]):
            documento= int(row["doc_prod"])
        else:
            documento=None
    else:
        if not pd.isna(row["doc_aso"]):
            documento= int(row["doc_aso"])
        else:
            documento=None
    response = {
        "documento_productor":documento,
        "poligono":str(row["geometry"]),
        "mongo_atribute":{
            "documento":str(documento),
            "nombre_productor":str(row["nom_prod"]),
            "area": row['area'],
            "nombre_finca": row['nomb_fin'],
            "fecha_visita": pd.to_datetime(row['fecha_vis']).strftime("%Y-%m-%d") if pd.notnull(row['fecha_vis']) else None
        }
    }
    response = limpiar_nan(response)
    return response

def reportar_conservacion(row):
    if pd.notna(row["doc_aso"]):
        documento = int(row["doc_aso"])
    elif pd.notna(row["doc_prod"]):
        documento = int(row["doc_prod"])
    else:
        documento = None

    response = {
        "documento_productor":documento,
        "poligono":str(row["geometry"]),
        "mongo_atribute":{
            "documento":str(documento),
            "nombre_productor":str(row["nom_prod"]),
            "area": row['area'],
            "tipo_arboles":row['tipo_arb'],
            "fecha_visita": pd.to_datetime(row['fecha_vis']).strftime("%Y-%m-%d") if pd.notnull(row['fecha_vis']) else None
        }
    }
    response = limpiar_nan(response)
    return response

def reportar_infraestructura(row):
    if pd.notna(row["doc_aso"]):
        documento = int(row["doc_aso"])
    elif pd.notna(row["doc_prod"]):
        documento = int(row["doc_prod"])
    else:
        documento = None
    response = {
        "documento_productor":documento,
        "poligono":str(row["geometry"]),
        "mongo_atribute":{
            "documento":str(documento),
            "nombre_productor":str(row["nom_prod"]),
            "area": row['area'],
            "fecha_visita": pd.to_datetime(row['fecha_vis']).strftime("%Y-%m-%d") if pd.notnull(row['fecha_vis']) else None,
            "tipo_estructura": row['tipo_estr'],
            "estructura": row['estruc_sel']
        }
    }
    response = limpiar_nan(response)
    return response

def actualizar_fincas_existentes(row):
    respuesta={
        "poligono":str(row["geometry"]),
        "mongo_atribute":{
            "documento":row["doc_aso"],
            "nombre_finca":row["nombre_fin"],
            "fecha_visita":pd.to_datetime(row['fecha_vis']).strftime("%Y-%m-%d") if pd.notnull(row['fecha_vis']) else None,
            "area":row["area"]
        }
    }
    response=limpiar_nan(respuesta)
    print(response)
    return response

def actualizar_lotes_existentes(row):
    respuesta = {
        "poligono":str(row["geometry"]),
        "mongo_atribute":{
                "numero_documento": str(row["doc_aso"]),
                "nombre_productor": str(row['nom_prod']),
                "fecha_visita": pd.to_datetime(row['fecha_vis']).strftime("%Y-%m-%d") if pd.notnull(row['fecha_vis']) else None,
                "area": row['area'],
                "observaciones": "",
                "descripcion": "",
                "numero_lote": None,
                "nombre_pol":row["nombre_pol"],
                "descripcion_lote": "",
                "variedad": row['variedad'],
                "distancia_surcos": row['dist_surco'],
                "distancia_plantas": row['dist_plant'],
                "densidad": row['densidad'],
                "numero_plantas": row['num_planta'],
                "gramos_plantas": row['gr_plant'],
                "kg_produccion": row['kg_produc'],
                "fecha_actividad": pd.to_datetime(row['fecha_acti']).strftime("%Y-%m-%d") if pd.notnull(row['fecha_acti']) else None,
                "produccion": row['produccion'],
                "estado_cultivo": row.get('estad_cult') if 'estad_cult' in row and pd.notnull(row['estad_cult']) else None,
                "subtipo_operacion": row['subtipo_op']
            }
    }
    respuesta = limpiar_nan(respuesta)
    return respuesta

In [ ]:

def reporte_coronel(gdf, tipo, metodo_reporte):
    respuestas_exitosas_lote = []
    respuestas_no_exitosas_lote = []
    for i, row in gdf.iterrows():
        respuesta = metodo_reporte(row)
        print(respuesta)
        response = session.post(capas_gdf_dict[tipo], json=respuesta, verify=False)
        print(response.status_code)
        print(response.json())
        if response.ok:
            try:
                data = response.json()
                # Solo guardar si la respuesta incluye un ID o estado positivo
                if "id" in data or data.get("status") == "success":
                    respuestas_exitosas_lote.append(data)
            except Exception as e:
                print(f"Error al procesar JSON en fila {i} del tipo {tipo}: {e}")
        else:
            respuestas_no_exitosas_lote.append(response.json())
    df_exitoso = pd.DataFrame(respuestas_exitosas_lote)
    df_no_exitoso = pd.DataFrame(respuestas_no_exitosas_lote)

    with pd.ExcelWriter(os.path.join(folder_base,f'{tipo[:-4]}.xlsx'), engine='openpyxl') as writer:
        df_exitoso.to_excel(writer, sheet_name='rep_suc', index=False)
        df_no_exitoso.to_excel(writer, sheet_name='rep_not_suc', index=False)


def actualizacion_coronel(gdf, tipo, metodo_reporte):
    
    respuestas_exitosas_lote = []
    respuestas_no_exitosas_lote = []
    gdf = gdf.loc[gdf["Actualizar"] == 1]
    url = capas_gdf_dict_agrupados[tipo]
    gdf = gdf.rename(columns={'id_1': 'id'})
    print(gdf)
    for i, row in gdf.iterrows():
        if pd.isna(row["id"]):
            respuestas_exitosas_lote.append({"objeto":row, "razon": "lote creado en capa erronea"})
        else:
            fecha_ahora = datetime.now()
            fecha_formateada = fecha_ahora.strftime("%d-%m-%Y_%H--%M--%S")
            url_actualizacion = f"{url}update/mongo_update/{row['id']}/{fecha_formateada}/"
            respuesta = metodo_reporte(row)
            response = session.post(url_actualizacion, json=respuesta, verify=False)
            respuesta["id"] = row["id"]
            print(response.ok)
            print(response.text)
            print(response.json())
            if response.ok:
                respuestas_exitosas_lote.append(respuesta)
            else:
                respuestas_no_exitosas_lote.append({"id":row['id'],"respuesta":respuesta})
    df_exitoso = pd.DataFrame(respuestas_exitosas_lote)
    df_no_exitoso = pd.DataFrame(respuestas_no_exitosas_lote)

    with pd.ExcelWriter(os.path.join(folder_base,f'{tipo[:-4]}.xlsx'), engine='openpyxl') as writer:
        df_exitoso.to_excel(writer, sheet_name='rep_suc', index=False)
        df_no_exitoso.to_excel(writer, sheet_name='rep_not_suc', index=False)

for tipo, ruta in files_individuales:
    if tipo in list(capas_gdf_dict.keys()):
        gdf = gpd.read_file(ruta)            
        # if tipo == "Lote.shp":
        #     reporte_coronel(gdf, tipo, reportar_lotes)
        if tipo == "Finca.shp":
            reporte_coronel(gdf, tipo, reportar_fincas)
        # if tipo == "Conservacion.shp":
        #     reporte_coronel(gdf, tipo, reportar_conservacion)
        # if tipo == "Construcciones.shp":
        #     reporte_coronel(gdf, tipo, reportar_infraestructura)

# for tipo, ruta in files_agrupados:
#     if tipo in list(capas_gdf_dict_agrupados.keys()):
#         gdf = gpd.read_file(ruta)
#         if tipo == "Fincas.shp":
#             actualizacion_coronel(gdf, tipo, actualizar_fincas_existentes)
#         if tipo.startswith("Lote"):
#             gdf["gr_plant"] = pd.to_numeric(gdf["gr_plant"], errors="coerce")
#             gdf["num_planta"] = pd.to_numeric(gdf["num_planta"], errors="coerce")
#             gdf["kg_produc"] = (gdf["gr_plant"]/1000)*gdf["num_planta"]
#             actualizacion_coronel(gdf, tipo, actualizar_lotes_existentes)